## Implementation of Camera Calibration through OpenCV

In [ ]:
# Imports
import cv2
import numpy as np
import glob

## Define dimensions of checkerboard

In [ ]:
# Defining the dimensions of checkerboard
 # 9,6 to begin with

# Each tile is 7x7 mm
chekcerboard_tile_size = 7 #mm

CHECKERBOARD = (4,4) # Vertical, horizontal
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

## Define world coordinates for 3D points

In [ ]:
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

## Extract path of individual image stored in ChessBoards folder

In [ ]:
# Need to calibrate with more than one image ideally. 
images = glob.glob('./Images/ChessBoards/LimitedBoards4/*.bmp') 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected, we refine the pixel coordinates and display them on the images of checker board
    """
    
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()

h,w = img.shape[:2]
 

## Camera calibration using OpenCV
* Zhang based calibration


In [ ]:
"""
Performing camera calibration by passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the detected corners (imgpoints)
"""

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix : \n") # Intrinsic parameters
print(mtx)
print("\n dist : \n") # [k1, k2, p1, p2, k3] k:radial distortion, p: tangential distortion
print(dist)
print("\n rvecs : \n") # Extrinsic Rodriguez
print(rvecs)
print("\n tvecs : \n") # Extrinsic Translation vector
print(tvecs)

print("tvecs[0] ", tvecs[0])


## Refine camera matrix and undistort images

In [ ]:
image_to_undistort = cv2.imread(images[0])

# Refining the camera matrix:
refined_camera_matrix, region_of_interest = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
print("Refined matrix: \n", refined_camera_matrix)

# Undistortion method 1:
distortion_1 = cv2.undistort(image_to_undistort, mtx, dist, None, refined_camera_matrix)
cv2.imshow("OG image: ", image_to_undistort)
cv2.imshow("Undistorted image, method 1: ", distortion_1)

# Undistortion method 2:
mapx, mapy =  cv2.initUndistortRectifyMap(mtx, dist, None, refined_camera_matrix, (w,h), 5)
distortion_2 = cv2.remap(image_to_undistort, mapx, mapy, cv2.INTER_LINEAR)
cv2.imshow("Undistorted image, method 2: ", distortion_2)
cv2.waitKey(0)

## Calculate re-projection error

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    image_points_2, _ = cv2.projectPoints(objpoints[i],rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], image_points_2, cv2.NORM_L2)/len(image_points_2)
    mean_error += error
    
mean_error = mean_error/len(objpoints)
print("Total mean error: ", mean_error)

print("RET from cv2.calibrateCamera: ", ret)

In [ ]:
# Invert camera matrix:
np.linalg.inv(mtx)
print("Inverted camera matrix: \n")
print(mtx) 

## Converting Rodrigues to rotational matrix

In [ ]:
rotation_matrix = np.zeros(shape=(3,3))
cv2.Rodrigues(rvecs[0], rotation_matrix)
print("Rotation matrix: \n", rotation_matrix)

rotation_mask = np.array([[1, 0], [0, 1], [0, 0]])

chopped_rotation = np.matmul(rotation_matrix, rotation_mask)
# print("\nChopped matrix: \n", chopped_rotation)

extrinsic_matrix = np.hstack((chopped_rotation,tvecs[0]))
# print("\nExtrinsic matrix: \n", extrinsic_matrix)

homography_matrix = np.matmul(mtx, extrinsic_matrix)
print("\nPre homography matrix: \n", homography_matrix)

homography_matrix = homography_matrix/homography_matrix[2][2]
print("\nPost homography matrix: \n", homography_matrix)

inv_homography_matrix = np.linalg.inv(homography_matrix)
print("\n Inverse homography matrix: \n", inv_homography_matrix)

pixels = np.array((1,2,1))
pixels = pixels.reshape(1,-1).transpose()
print("Pixels: \n", pixels)

world_coordinates = inv_homography_matrix * pixels
print("\nWorld Coordinates: \n", world_coordinates)


In [ ]:
# Function for legible outputs:

def calculateWorldCoordinates(pixels_to_calculate, intrinsicMatrix, extrinsicMatrix):
    inverse_intrinsic = np.linalg.inv(intrinsicMatrix)
    inverse_extrinsic = np.linalg.inv(extrinsicMatrix)
    init_coords = np.dot(inverse_extrinsic, inverse_intrinsic)
    world_coordinate_vector = np.dot(init_coords, pixels_to_calculate)
    world_coordinate_vector = world_coordinate_vector/world_coordinate_vector[2]
    return world_coordinate_vector

pix1 = [0,0,1]
pix2 = [3,0,0]
pix3 = objp[0][12]
pixels = [pix1, pix2, pix3]

def coordinatesToCSV(pixels_to_calculate, intrinsicMatrix, extrinsicMatrix, outputFile):
    # pixelCoordinates = pixelCoordinates.reshape(1,-1).transpose()
    ok = True
    world_coordinate_list = []
    try: 
        outputFile = open(outputFile, "w")
    except:
        ok = False
        print("Unable to open output file.")
    if ok:
        outputFile.write("World coordinates are on the form [X, Y, Z, 1], where the '1' is left out for this file.\n")
        for pixelCount,pixelCoordinate in enumerate(pixels_to_calculate):
            outputFile.write("\nPixel count: " + str(pixelCount) + "\nPixel coordinates: " + str(pixelCoordinate) + "\n")
            worldCoordinates = calculateWorldCoordinates(pixelCoordinate, intrinsicMatrix, extrinsicMatrix )
            world_coordinate_list.append(worldCoordinates)
            outputFile.write("\nWorld coordinates: \n" + str(worldCoordinates) + "\n")
    
    print("Written to file: ", outputFile)
    outputFile.close()
    return world_coordinate_list

outputFile = "Coordinates.csv"
world_coordinate_list = coordinatesToCSV(pixels, mtx, extrinsic_matrix, outputFile)
